In [1]:
import math
import uproot
import numpy as np
import matplotlib.pyplot as plt

In [3]:
BASEDIR = '/depot/cms/top/bakshi3/TopSpinCorr_Run2_generalized_ND/CMSSW_10_6_30/src/TopAnalysis/Configuration/analysis/diLeptonic/TUnfoldResults_2016/Nominal/combined/AllSysts_2D_vars_updated_reg/'

In [4]:
var_list = [
'b1k_mttbar',
'b2k_mttbar',
'b1r_mttbar',
'b2r_mttbar',
'b1n_mttbar',
'b2n_mttbar',
'b1j_mttbar',
'b2j_mttbar',
'b1q_mttbar',
'b2q_mttbar',
'c_kk_mttbar',
'c_rr_mttbar',
'c_nn_mttbar',
'c_Prk_mttbar',
'c_Mrk_mttbar',
'c_Pnr_mttbar',
'c_Mnr_mttbar',
'c_Pnk_mttbar',
'c_Mnk_mttbar',
'c_kj_mttbar',
'c_rq_mttbar',
'c_han_mttbar',
'c_tra_mttbar',
'c_sca_mttbar',
'c_Prj_mttbar',
'c_Mrj_mttbar',
'c_kjL_mttbar',
'c_rqL_mttbar',
'c_rkP_mttbar',
'c_rkM_mttbar',
'c_nrP_mttbar',
'c_nrM_mttbar',
'c_nkP_mttbar',
'c_nkM_mttbar',
'll_cHel_mttbar',
'll_cLab_mttbar',
'llbar_delta_phi_mttbar',
'llbar_delta_eta_mttbar',
]

In [5]:
fileptr_dict = {}
for var in var_list :
    fileptr_dict[var] = uproot.open(BASEDIR + str(var) + ".root")

In [6]:
[k for k in fileptr_dict['b1k_mttbar'].keys() if 'RespMatSys_Uncorr_CovMatSys' in k]

['b1k_mttbarRespMatSys_Uncorr_CovMatSys;2',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSys;1',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSys_rebinnedA;2',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSys_rebinnedA;1',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSys_rebinnedB;2',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSys_rebinnedB;1',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSysNorm;2',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSysNorm;1',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSysNorm_rebinnedA;2',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSysNorm_rebinnedA;1',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSysNorm_rebinnedB;2',
 'b1k_mttbarRespMatSys_Uncorr_CovMatSysNorm_rebinnedB;1']

## Load in statistical uncertainty for Allvars

In [7]:
nbinsvar = 24
deltaStatAllVar = []

for var in var_list :
    # Statistical uncertainty from unfolding 
    statcovmat   = fileptr_dict[var][var + 'EmatrixCor_rebinnedA'].to_numpy()[0]
    
    # Statistical uncertainty from limited MC stats
    MCstatcovmat = fileptr_dict[var][var + 'RespMatSys_Uncorr_CovMatSys_rebinnedA'].to_numpy()[0]

    # Add both 
    deltaStatVar = [math.sqrt(statcovmat[i][i] + MCstatcovmat[i][i]) for i in range(nbinsvar)]
    
    deltaStatAllVar.extend(deltaStatVar)
    
    del statcovmat
    del MCstatcovmat
    del deltaStatVar

### Compute matrix with sigma along it's diagnols

In [8]:
deltaStatAllVar   = np.array(deltaStatAllVar)
diag_sigma_matrix = np.zeros((len(deltaStatAllVar), len(deltaStatAllVar)))

for i in range(len(deltaStatAllVar)) :
    diag_sigma_matrix[i][i] = deltaStatAllVar[i]

In [9]:
selfCorrMatrix = np.loadtxt('RhoBBCorrected_rebinnedA_updated_reg.txt')
selfCovMatrix  = np.matmul(np.matmul(diag_sigma_matrix, selfCorrMatrix), diag_sigma_matrix.T)

In [10]:
np.shape(selfCorrMatrix)

(912, 912)

In [14]:
np.savetxt('StatCovMatrix_2D_mttbar_updated_reg.txt', selfCovMatrix)

### Compare to the one obtained from the FW

In [13]:
SystematicsAllVars   = uproot.open(BASEDIR + "Systematics_AllVars.root")
# SystematicsAllVars = uproot.open(BASEDIR + "Systematics_AllVars_incomplete.root")

In [14]:
[k for k in SystematicsAllVars.keys() if 'TotalStat' in k]

['TotalStatCovMatrix_AllVar_rebinnedA;1',
 'TotalStatSystCovMatrix_AllVar_rebinnedA;1']

In [15]:
FWCovMatrix = SystematicsAllVars['TotalStatCovMatrix_AllVar_rebinnedA'].to_hist().to_numpy()[0]
FWCovMatrix = np.round(FWCovMatrix,5)
FWCovMatrix

array([[ 1.341663e+01, -1.918470e+00, -8.126140e+00, ...,  2.420800e-01,
         1.812000e-02,  5.844000e-02],
       [-1.918470e+00,  5.296590e+00,  2.669470e+00, ...,  1.409000e-02,
         3.390000e-02,  3.416000e-02],
       [-8.126140e+00,  2.669470e+00,  1.169248e+01, ..., -1.499900e-01,
         1.290900e-01, -3.828000e-02],
       ...,
       [ 2.420800e-01,  1.409000e-02, -1.499900e-01, ...,  2.732400e-01,
        -1.780000e-03,  7.929000e-02],
       [ 1.812000e-02,  3.390000e-02,  1.290900e-01, ..., -1.780000e-03,
         3.017400e-01,  5.170000e-03],
       [ 5.844000e-02,  3.416000e-02, -3.828000e-02, ...,  7.929000e-02,
         5.170000e-03,  1.143270e+00]])

In [16]:
selfCovMatrix = np.round(selfCovMatrix,5)
selfCovMatrix

array([[ 1.341663e+01, -1.356830e+00, -7.720440e+00, ...,  3.757000e-01,
        -2.564000e-02, -5.615100e-01],
       [-1.356830e+00,  5.296590e+00,  3.268120e+00, ...,  2.704000e-02,
         2.368600e-01,  2.204600e-01],
       [-7.720440e+00,  3.268120e+00,  1.169248e+01, ..., -3.457700e-01,
         1.580000e-03,  7.412400e-01],
       ...,
       [ 3.757000e-01,  2.704000e-02, -3.457700e-01, ...,  2.732400e-01,
         1.051000e-02, -1.795000e-02],
       [-2.564000e-02,  2.368600e-01,  1.580000e-03, ...,  1.051000e-02,
         3.017400e-01, -9.720000e-03],
       [-5.615100e-01,  2.204600e-01,  7.412400e-01, ..., -1.795000e-02,
        -9.720000e-03,  1.143270e+00]])

In [17]:
selfdiag = np.array([selfCovMatrix[i][i] for i in range(864)])
FWdiag   = np.array([FWCovMatrix[i][i] for i in range(864)])

(selfdiag == FWdiag).all()

True

### Also compare correlations while you are at it

In [125]:
np.round(selfCorrMatrix,5)

array([[ 1.     , -0.3067 , -0.08428, ...,  0.02417, -0.01229,  0.02498],
       [-0.3067 ,  1.     , -0.19079, ...,  0.08526,  0.01861,  0.05664],
       [-0.08428, -0.19079,  1.     , ...,  0.06688,  0.05704,  0.02915],
       ...,
       [ 0.02417,  0.08526,  0.06688, ...,  1.     , -0.05175,  0.00524],
       [-0.01229,  0.01861,  0.05704, ..., -0.05175,  1.     ,  0.01988],
       [ 0.02498,  0.05664,  0.02915, ...,  0.00524,  0.01988,  1.     ]])

In [126]:
DEPOTDIR    = '/depot/cms/top/bakshi3/TopSpinCorr_Run2_generalized_ND/CMSSW_10_6_30/src/TopAnalysis/Configuration/analysis/diLeptonic/'
FW_1000_cor = uproot.open(DEPOTDIR  + 'BootstrapCorrelationMatrices_1D_newvars_1000PE_FW.root')['h_RhoBBCorrected_rebinnedA']
FW_1000_cor = FW_1000_cor.to_hist().to_numpy()[0]

np.round(FW_1000_cor,5)

array([[ 1.     , -0.3067 , -0.08427, ...,  0.0242 , -0.01233,  0.02498],
       [-0.3067 ,  1.     , -0.19082, ...,  0.08533,  0.0186 ,  0.05659],
       [-0.08427, -0.19082,  1.     , ...,  0.06685,  0.05709,  0.02977],
       ...,
       [ 0.0242 ,  0.08533,  0.06685, ...,  1.     , -0.05167,  0.00566],
       [-0.01233,  0.0186 ,  0.05709, ..., -0.05167,  1.     ,  0.01947],
       [ 0.02498,  0.05659,  0.02977, ...,  0.00566,  0.01947,  1.     ]])

In [127]:
# Jacob mentioned using delta instead of rel_delta 
delta = (FW_1000_cor - selfCorrMatrix)
delta

# There exist no elements where absolute delta is greater than 0.001
# which is the statistical precision of 10k toys
np.where(abs(delta) > 0.001)

(array([], dtype=int64), array([], dtype=int64))

### The fact that the correlations are the same to about 0.001 and that the diagnols are the same should probably serve as an indicator to the fact that the method works

In [131]:
selfdiag.all() == FWdiag.all()

True

## Now let's normalize

Consider an observable $x$ and its covariance matrix $\Sigma_x$. The generalized covariance transformation is $\Sigma_y = J \Sigma_x J^T$ where $J$ is the Jacobian matrix:

$J = 
\begin{bmatrix} 
\partial f_0 / \partial x_0 & \partial f_0 / \partial x_1 & \ldots \\
\partial f_1 / \partial x_0 & \partial f_1 / \partial x_1 & \ldots \\
\ldots & & \\
\end{bmatrix}$

Now consider a transformation of $x$ to normalize it: 

$y = \frac{x}{ \sum_i x_i} \equiv \frac{x}{N}$,


Then the Jacobian is,

$J = 
\begin{bmatrix} 
\frac{N-x_0}{N^2}  & \frac{-x_0}{N^2} & \frac{-x_0}{N^2} & \ldots \\
\frac{-x_1}{N^2} & \frac{N-x_1}{N^2} & \frac{-x_1}{N^2} & \ldots \\
\ldots 
\end{bmatrix}$


Coordinate transformations change the uncertainties as,

$\Sigma_y = J \Sigma_x J^T$

where $\sigma_x$ is the covariance matrix, and $y$ is defined above. 

In [15]:
nbinsrhoi   = 24
nbinstot    = nbinsrhoi * len(var_list)
sum_AllVar  = []
hnom_AllVar = []

for var in var_list : 

    # Unfolded result for particular variable
    hnom = fileptr_dict[var][var + 'TUnfResultCor_rebinnedA'].to_hist().values()
    sum_AllVar.append(sum(hnom))
    hnom_AllVar.append(hnom)
    
dfdn  = np.zeros((nbinstot, nbinstot))

for i in range(nbinstot) :
    for j in range(nbinstot) :
        if (i == j) : 
            sum_n =  sum_AllVar[i//nbinsrhoi] # i//nbinsrhoi is var number, so get integral for that variable
            unf_i = hnom_AllVar[i//nbinsrhoi][i%nbinsrhoi] # i%nbinsrhoi is bin number, so list <var> <bin>
            
            dfdn[i][j]  = (sum_n - unf_i) / sum_n**2
            del sum_n, unf_i
            
        elif (i//nbinsrhoi == j//nbinsrhoi) :
            sum_n =  sum_AllVar[i//nbinsrhoi] # i//nbinsrhoi is var number, so get integral for that variable
            unf_i = hnom_AllVar[i//nbinsrhoi][i%nbinsrhoi] # i%nbinsrhoi is bin number, so list <var> <bin>
            
            dfdn[i][j]  = (- unf_i) / sum_n**2
            del sum_n, unf_i
            
        else :
            dfdn[i][j] = 0

In [17]:
112//6 # var_number

18

In [18]:
112%6 # bin_number

4

In [16]:
np.matmul(np.matmul(dfdn, selfCovMatrix), dfdn.T)

array([[ 1.91373832e-05, -2.20910598e-06, -1.13752596e-05, ...,
         5.12753954e-07, -3.69187025e-08, -8.12118070e-07],
       [-2.20910598e-06,  7.76135509e-06,  4.92134836e-06, ...,
         1.42905603e-08,  3.61188915e-07,  3.70762950e-07],
       [-1.13752596e-05,  4.92134836e-06,  1.73699650e-05, ...,
        -5.18211652e-07,  3.06643768e-08,  1.18018610e-06],
       ...,
       [ 5.12753954e-07,  1.42905603e-08, -5.18211652e-07, ...,
         4.20169020e-07,  4.09974488e-08,  7.22351633e-09],
       [-3.69187025e-08,  3.61188915e-07,  3.06643768e-08, ...,
         4.09974488e-08,  4.82008598e-07,  4.62718303e-08],
       [-8.12118070e-07,  3.70762950e-07,  1.18018610e-06, ...,
         7.22351633e-09,  4.62718303e-08,  1.75002156e-06]])

In [17]:
# FWCovMatrix_norm = SystematicsAllVars['TotalStatCovMatrix_AllVarNorm_rebinnedA'].to_hist().to_numpy()[0]
# FWCovMatrix_norm

In [18]:
selfCovMatrixNorm = np.matmul(np.matmul(dfdn, selfCovMatrix), dfdn.T)
selfCovMatrixNorm

array([[ 1.91373832e-05, -2.20910598e-06, -1.13752596e-05, ...,
         5.12753954e-07, -3.69187025e-08, -8.12118070e-07],
       [-2.20910598e-06,  7.76135509e-06,  4.92134836e-06, ...,
         1.42905603e-08,  3.61188915e-07,  3.70762950e-07],
       [-1.13752596e-05,  4.92134836e-06,  1.73699650e-05, ...,
        -5.18211652e-07,  3.06643768e-08,  1.18018610e-06],
       ...,
       [ 5.12753954e-07,  1.42905603e-08, -5.18211652e-07, ...,
         4.20169020e-07,  4.09974488e-08,  7.22351633e-09],
       [-3.69187025e-08,  3.61188915e-07,  3.06643768e-08, ...,
         4.09974488e-08,  4.82008598e-07,  4.62718303e-08],
       [-8.12118070e-07,  3.70762950e-07,  1.18018610e-06, ...,
         7.22351633e-09,  4.62718303e-08,  1.75002156e-06]])

In [19]:
np.savetxt('StatCovMatrixNorm_2D_mttbar_updated_reg.txt', selfCovMatrixNorm)

In [20]:
selfCovMatrixNorm

array([[ 1.91373832e-05, -2.20910598e-06, -1.13752596e-05, ...,
         5.12753954e-07, -3.69187025e-08, -8.12118070e-07],
       [-2.20910598e-06,  7.76135509e-06,  4.92134836e-06, ...,
         1.42905603e-08,  3.61188915e-07,  3.70762950e-07],
       [-1.13752596e-05,  4.92134836e-06,  1.73699650e-05, ...,
        -5.18211652e-07,  3.06643768e-08,  1.18018610e-06],
       ...,
       [ 5.12753954e-07,  1.42905603e-08, -5.18211652e-07, ...,
         4.20169020e-07,  4.09974488e-08,  7.22351633e-09],
       [-3.69187025e-08,  3.61188915e-07,  3.06643768e-08, ...,
         4.09974488e-08,  4.82008598e-07,  4.62718303e-08],
       [-8.12118070e-07,  3.70762950e-07,  1.18018610e-06, ...,
         7.22351633e-09,  4.62718303e-08,  1.75002156e-06]])